In [1]:
import tensorflow as tf
import numpy as np

In [311]:
train_data_file = "D:/UCSD/F17/CSE293/data_prep_scripts/train_data.npy"
train_data = np.load(train_data_file)

In [312]:
class model(object):
    
    def __init__(self, ckpt_path, lr, epochs=1000, dim_lang=5, model_name="MLPM"):
        self.epochs = epochs
        self.ckpt_path = ckpt_path
        self.model_name = model_name
        self.lr = lr
        self.dim_lang = dim_lang
        
        self.all_lang_rep = dict()
                
        for lang in ["en", "pt", "es"]:
            self.all_lang_rep[lang] = np.random.normal(loc=0.0, scale=1/np.sqrt(float(dim_lang)), size=(dim_lang))
        
        def __graph__():
            tf.reset_default_graph()
            self.graph = tf.Graph()
            with self.graph.as_default():
                
                # source and target vector representation
                self.source_words = tf.placeholder(tf.float32, shape=[None, 300])
                self.target_words = tf.placeholder(tf.float32, shape=[None, 300])
                
                # parameter matrices
                self.encoder = tf.Variable(tf.truncated_normal([300, 300],\
                                           stddev=1/tf.sqrt(300.0)), name='encoder')
                
                self.decoder = tf.Variable(tf.truncated_normal([300, 300],\
                                           stddev=1/tf.sqrt(300.0)), name='decoder')
                
                self.lang_encoder = tf.Variable(tf.truncated_normal([2*dim_lang,300],\
                                               stddev=1/tf.sqrt(300.0)), name='lang_encoder')

                # language representation
                self.lang_rep_placeholder = tf.placeholder(tf.float32, shape=[1,2*self.dim_lang])
                self.lang_rep = tf.get_variable("lang_rep_in",shape=[1,2*self.dim_lang], dtype=tf.float32)
                                
                self.assign_lang_rep_op = self.lang_rep.assign(self.lang_rep_placeholder).op 
                # model equation
                self.target_pred = self.get_model(self.encoder, self.decoder, self.source_words,\
                                                  self.lang_rep, self.lang_encoder)
                
                #squared loss
                self.loss = tf.reduce_sum(tf.square(self.target_words-self.target_pred))
                
                #self.train_step = tf.train.GradientDescentOptimizer(self.lr).minimize(self.loss)
                self.train_step = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
                self.init = tf.global_variables_initializer()
                self.saver = tf.train.Saver()
        
        print('start building graph')
        __graph__()
        print('graph built')
        
    # encoder, decoder: 300x300   source_words: Nonex300    lang_rep: 1x2*dim_lang    lang_encoder: 2*dim_langx300
    # Output: Nonex300
    def get_model(self, encoder, decoder, source_words, lang_rep, lang_encoder):
        
        shared_source_words = tf.matmul(source_words, encoder)
        shared_lang_rep = tf.matmul(lang_rep, lang_encoder)
        
        num_examples = tf.shape(shared_source_words)[0]
        #(shared_source_words.get_shape())[0]
        tile = tf.tile(shared_lang_rep, tf.convert_to_tensor([1, num_examples]))
        shared_lang_aux = tf.reshape(tile, [num_examples, 300])
        shared_embedding_vector = shared_lang_rep+shared_lang_aux
        
        ret = tf.matmul(shared_embedding_vector, decoder)
        return ret 
    
    def get_feed(self, X, Y, src_lang, dest_lang):
        feed_dict = {self.source_words: X, self.target_words: Y}       
        return feed_dict
    
    #train -> train[en_pt]
    def train(self, train, train_lang_pairs, batch_size, validation, num_epochs=10, sess=None):
        if sess == None:
            sess = tf.Session(graph = self.graph)
            sess.run(self.init)
            
        with sess.as_default():
            assert sess is tf.get_default_session()
            print (self.lang_rep.eval())
        #max data for a language pair/batch size
        max_data_size = int(max([len(x) for x in train])/batch_size)
    
        for epoch in range(num_epochs):
            batch_index = np.zeros(shape=(len(train)), dtype=int)
            for batch_number in range(max_data_size):
                #make batches
                for lang_pair_data_enum in enumerate(train):
                    i, lang_pair_data = lang_pair_data_enum
                    lang_pair_data = np.array(lang_pair_data)
                    print(np.shape(lang_pair_data))
                    if batch_index[i] >= len(lang_pair_data):
                        batch_index[i] = 0
                    print(batch_index[i])
                    cur_batch = lang_pair_data[batch_index[i]:(batch_index[i]+batch_size),:,:]
                    batch_index[i] += batch_size
                    print(np.shape(cur_batch))
                    X = cur_batch[:,0,:]
                    Y = cur_batch[:,1,:]
                    print(np.shape(X))
                    print(np.shape(Y))
                    lang = train_lang_pairs[i].split('-')
                    
                    A = self.all_lang_rep[lang[0]]
                    B = self.all_lang_rep[lang[1]]
                    init = np.insert(B, np.arange(len(A)), A)
                    init = init.reshape(1, np.shape(init)[0])
                    
                    sess.run(self.assign_lang_rep_op, feed_dict={self.lang_rep_placeholder : init})

                    #batch index more that lang pair data length not handled :P
                    _, train_loss, lang_rep = sess.run([self.train_step, self.loss, self.lang_rep], self.get_feed(X, Y, lang[0], lang[1]))
                    
                    
                    print(lang_rep)
                    print(self.lang_rep.get_shape())
                    self.all_lang_rep[lang[0]], self.all_lang_rep[lang[1]] = lang_rep[0][:5], lang_rep[0][5:]
                    
                    print("Batch:" + str(batch_number))
                    print("Loss:" + str(train_loss))
                    print(lang)
                    print("------------------------------")
            
            #save epoch
            if epoch and epoch%10==0:
                self.saver.save(sess, self.ckpt_path+self.model_name+".ckpt", global_step=epoch)
                
                #print losses #todo: format for train data with lang
        self.saver.save(sess, self.ckpt_path+self.model_name+".ckpt", global_step=num_epochs+1)
        np.savetxt()



In [315]:
mlpm = model(ckpt_path="./ckpt/",lr=0.01, epochs=1)

start building graph
graph built


In [316]:
mlpm.train(train_data,train_lang_pairs= ["pt-es", "pt-en", "en-es"], batch_size=40, validation=[])

[[ 0.70915085  0.36378402 -0.29474384 -0.50978178 -0.40561798 -0.08899784
  -0.21721905  0.66658133  0.41962367 -0.58315432]]
(303, 2, 300)
0
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666 -0.99137229 -0.83516777 -0.36235678  0.34606239  0.03933046
   0.50970888 -0.19796824 -0.11783596 -0.11286956]]
(1, 10)
Batch:0
Loss:840.468
['pt', 'es']
------------------------------
(415, 2, 300)
0
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.02415964 -0.99137229  0.40884572 -0.83516777  0.20843136
  -0.36235678  0.15552697  0.34606239  0.66683775]]
(1, 10)
Batch:0
Loss:1050.77
['pt', 'en']
------------------------------
(151632, 2, 300)
0
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.20843136  0.03933046 -0.36235678  0.50970888  0.15552697 -0.19796824
   0.34606239 -0.11783596  0.66683775 -0.11286956]]
(1, 10)
Batch:0
Loss:901.182
['en', 'es']
------------------------------
(303, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666 -0.19796824  0.02415964  0.34606239 -0.99137229 -0.11783596
   

(151632, 2, 300)
440
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.83516777  0.50970888  0.02415964  0.66683775 -0.19796824  0.34606239
   0.20843136  0.40884572 -0.99137229 -0.11286956]]
(1, 10)
Batch:11
Loss:2060.23
['en', 'es']
------------------------------
(303, 2, 300)
160
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.34606239 -0.11783596  0.20843136 -0.36235678  0.40884572
   0.15552697 -0.99137229  0.03933046 -0.11286956]]
(1, 10)
Batch:12
Loss:1014.46
['pt', 'es']
------------------------------
(415, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666 -0.83516777  0.34606239  0.50970888 -0.11783596  0.02415964
   0.20843136  0.66683775 -0.36235678 -0.19796824]]
(1, 10)
Batch:12
Loss:1007.46
['pt', 'en']
------------------------------
(151632, 2, 300)
480
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.02415964  0.40884572  0.20843136  0.15552697  0.66683775 -0.99137229
  -0.36235678  0.03933046 -0.19796824 -0.11286956]]
(1, 10)
Batch:12
Loss:1617.29
['en', 'es']
----------------------

(151632, 2, 300)
880
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.03933046  0.66683775 -0.11783596 -0.99137229  0.34606239  0.20843136
  -0.83516777  0.15552697 -0.36235678 -0.11286956]]
(1, 10)
Batch:22
Loss:1275.78
['en', 'es']
------------------------------
(303, 2, 300)
280
(23, 2, 300)
(23, 300)
(23, 300)
[[-0.16361666  0.20843136  0.40884572 -0.83516777  0.02415964  0.15552697
  -0.19796824 -0.36235678  0.50970888 -0.11286956]]
(1, 10)
Batch:23
Loss:505.796
['pt', 'es']
------------------------------
(415, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.03933046  0.20843136  0.66683775  0.40884572 -0.11783596
  -0.83516777 -0.99137229  0.02415964  0.34606239]]
(1, 10)
Batch:23
Loss:1532.97
['pt', 'en']
------------------------------
(151632, 2, 300)
920
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.11783596  0.15552697 -0.83516777 -0.19796824 -0.99137229 -0.36235678
   0.02415964  0.50970888  0.34606239 -0.11286956]]
(1, 10)
Batch:23
Loss:1020.2
['en', 'es']
-----------------------

(151632, 2, 300)
1320
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.50970888 -0.99137229  0.40884572 -0.36235678  0.20843136 -0.83516777
   0.03933046 -0.19796824  0.02415964 -0.11286956]]
(1, 10)
Batch:33
Loss:959.109
['en', 'es']
------------------------------
(303, 2, 300)
80
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666 -0.83516777  0.15552697  0.03933046 -0.11783596 -0.19796824
   0.34606239  0.02415964  0.66683775 -0.11286956]]
(1, 10)
Batch:34
Loss:888.436
['pt', 'es']
------------------------------
(415, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.50970888 -0.83516777 -0.99137229  0.15552697  0.40884572
   0.03933046 -0.36235678 -0.11783596  0.20843136]]
(1, 10)
Batch:34
Loss:816.91
['pt', 'en']
------------------------------
(151632, 2, 300)
1360
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.40884572 -0.19796824  0.03933046  0.34606239 -0.36235678  0.02415964
  -0.11783596  0.66683775  0.20843136 -0.11286956]]
(1, 10)
Batch:34
Loss:1006.6
['en', 'es']
-----------------------

(151632, 2, 300)
1760
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.66683775 -0.36235678  0.15552697  0.02415964 -0.83516777  0.03933046
   0.50970888  0.34606239 -0.11783596 -0.11286956]]
(1, 10)
Batch:44
Loss:1144.26
['en', 'es']
------------------------------
(303, 2, 300)
200
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.03933046 -0.19796824  0.50970888  0.40884572  0.34606239
   0.20843136 -0.11783596 -0.99137229 -0.11286956]]
(1, 10)
Batch:45
Loss:784.457
['pt', 'es']
------------------------------
(415, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.66683775  0.03933046 -0.36235678 -0.19796824  0.15552697
   0.50970888  0.02415964  0.40884572 -0.83516777]]
(1, 10)
Batch:45
Loss:765.937
['pt', 'en']
------------------------------
(151632, 2, 300)
1800
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.15552697  0.34606239  0.50970888  0.20843136  0.02415964 -0.11783596
   0.40884572 -0.99137229 -0.83516777 -0.11286956]]
(1, 10)
Batch:45
Loss:1095.54
['en', 'es']
--------------------

...

   0.03933046 -0.36235678 -0.11783596  0.20843136]]
(1, 10)
Batch:242
Loss:854.532
['pt', 'en']
------------------------------
(151632, 2, 300)
9680
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.40884572 -0.19796824  0.03933046  0.34606239 -0.36235678  0.02415964
  -0.11783596  0.66683775  0.20843136 -0.11286956]]
(1, 10)
Batch:242
Loss:1072.87
['en', 'es']
------------------------------
(303, 2, 300)
120
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.02415964  0.50970888 -0.11783596 -0.83516777  0.66683775
  -0.99137229  0.20843136  0.15552697 -0.11286956]]
(1, 10)
Batch:243
Loss:676.153
['pt', 'es']
------------------------------
(415, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.40884572  0.02415964 -0.19796824  0.50970888  0.03933046
  -0.11783596  0.34606239 -0.83516777 -0.36235678]]
(1, 10)
Batch:243
Loss:767.13
['pt', 'en']
------------------------------
(151632, 2, 300)
9720
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.03933046  0.66683775 -0.11783596 -0.99137229  0.3460623

(151632, 2, 300)
10120
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.15552697  0.34606239  0.50970888  0.20843136  0.02415964 -0.11783596
   0.40884572 -0.99137229 -0.83516777 -0.11286956]]
(1, 10)
Batch:253
Loss:1381.02
['en', 'es']
------------------------------
(303, 2, 300)
240
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666 -0.11783596  0.66683775  0.40884572  0.03933046 -0.99137229
  -0.36235678 -0.83516777 -0.19796824 -0.11286956]]
(1, 10)
Batch:254
Loss:1171.65
['pt', 'es']
------------------------------
(415, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.15552697 -0.11783596  0.34606239  0.66683775  0.50970888
   0.40884572  0.20843136  0.03933046  0.02415964]]
(1, 10)
Batch:254
Loss:877.579
['pt', 'en']
------------------------------
(151632, 2, 300)
10160
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.50970888 -0.99137229  0.40884572 -0.36235678  0.20843136 -0.83516777
   0.03933046 -0.19796824  0.02415964 -0.11286956]]
(1, 10)
Batch:254
Loss:1030.45
['en', 'es']
--------------

(151632, 2, 300)
10560
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.19796824  0.20843136  0.66683775 -0.83516777 -0.11783596  0.40884572
   0.15552697 -0.36235678  0.03933046 -0.11286956]]
(1, 10)
Batch:264
Loss:1483.48
['en', 'es']
------------------------------
(303, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.40884572 -0.99137229  0.15552697  0.50970888 -0.36235678
   0.02415964  0.03933046  0.34606239 -0.11286956]]
(1, 10)
Batch:265
Loss:622.505
['pt', 'es']
------------------------------
(415, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666 -0.19796824  0.40884572  0.20843136 -0.99137229  0.66683775
   0.15552697 -0.83516777  0.50970888 -0.11783596]]
(1, 10)
Batch:265
Loss:788.074
['pt', 'en']
------------------------------
(151632, 2, 300)
10600
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.66683775 -0.36235678  0.15552697  0.02415964 -0.83516777  0.03933046
   0.50970888  0.34606239 -0.11783596 -0.11286956]]
(1, 10)
Batch:265
Loss:1205.18
['en', 'es']
---------------

(151632, 2, 300)
11000
(40, 2, 300)
(40, 300)
(40, 300)
[[ 0.34606239 -0.83516777 -0.99137229  0.03933046  0.40884572  0.15552697
  -0.19796824  0.02415964  0.50970888 -0.11286956]]
(1, 10)
Batch:275
Loss:934.517
['en', 'es']
------------------------------
(303, 2, 300)
160
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.15552697 -0.36235678 -0.19796824  0.66683775  0.02415964
  -0.11783596  0.50970888  0.20843136 -0.11286956]]
(1, 10)
Batch:276
Loss:863.307
['pt', 'es']
------------------------------
(415, 2, 300)
40
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.16361666  0.34606239  0.15552697 -0.83516777 -0.36235678 -0.99137229
  -0.19796824  0.03933046  0.66683775  0.40884572]]
(1, 10)
Batch:276
Loss:700.869
['pt', 'en']
------------------------------
(151632, 2, 300)
11040
(40, 2, 300)
(40, 300)
(40, 300)
[[-0.99137229  0.02415964 -0.19796824 -0.11783596  0.03933046  0.50970888
   0.66683775  0.20843136  0.40884572 -0.11286956]]
(1, 10)
Batch:276
Loss:852.373
['en', 'es']
--------------

KeyboardInterrupt: 